In [1]:
import matplotlib.pyplot as plt
import glob
import pickle
from collections import defaultdict
import pandas as pd
import seaborn as sns
from utils import *

/opt/conda/envs/Base/lib/python3.9/site-packages/sktime/utils/validation/series.py:21: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  VALID_INDEX_TYPES = (pd.Int64Index, pd.RangeIndex, pd.PeriodIndex, pd.DatetimeIndex)


In [2]:
pdf = get_problem_algorithm_performance('prediction_errors/mape.csv')
forecasting_algorithms = list(pdf.columns)

get_problem_algorithm_performance took=0:00:02.273947 shape=(100000, 61)


In [3]:
data = defaultdict(lambda: [])

files = glob.glob('trained_model/feature_compare/*/meta.p')
for file in files:
    meta = pickle.load(open(file, "rb"))
    algo = re_find('m_(.+?)_f_', file)
    features = re_find('f_(.+?)_r_', file)
    run = re_find('_r_(.+?)_c_', file)
    cor = re_find('c_(.+?)/', file)
    
    for i, fa_error in enumerate(meta['error_column_mse']):
        data['ForecastingAlgorithmError'].append(fa_error)
        data['ForecastingAlgorithm'].append(forecasting_algorithms[i])
        
        
        data['Model'].append(algo)
        data['Features'].append(features)

        data['Run'].append(run)
        data['FeatureCorrelation'].append(cor)
        
df = pd.DataFrame(dict(data))
df

,ForecastingAlgorithmError,ForecastingAlgorithm,Model,Features,Run,FeatureCorrelation
0,0.010417,250,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",1,0.6
1,0.009663,118,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",1,0.6
2,0.010168,238,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",1,0.6
3,0.010770,039,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",1,0.6
4,0.010080,072,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",1,0.6
...,...,...,...,...,...,...
134988,0.013877,234,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",4,0.8
134989,0.010023,SES,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",4,0.8
134990,0.011472,255,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",4,0.8
134991,0.013107,082,M4KerasNetRegressionModel,"TSFresh,TSFreshDiff,TSFreshLog",4,0.8


In [18]:
df.groupby(['Model', 'ForecastingAlgorithm', 'Features', 'FeatureCorrelation']).count().reset_index().query(f'Run == 30')

,Model,ForecastingAlgorithm,Features,FeatureCorrelation,ForecastingAlgorithmError,Run
14,M4DummyMeanRegression,005,TSFresh,0.5,30,30
49,M4DummyMeanRegression,009,TSFresh,0.5,30,30
84,M4DummyMeanRegression,024,TSFresh,0.5,30,30
119,M4DummyMeanRegression,030,TSFresh,0.5,30,30
154,M4DummyMeanRegression,036,TSFresh,0.5,30,30
...,...,...,...,...,...,...
14784,M4XGBRegressor,Naive2,TSFresh,0.5,30,30
14819,M4XGBRegressor,RNN,TSFresh,0.5,30,30
14854,M4XGBRegressor,SES,TSFresh,0.5,30,30
14889,M4XGBRegressor,Theta,TSFresh,0.5,30,30


In [13]:
df.query(f'ForecastingAlgorithm == "250" and Model == "M4KerasNetRegressionModel" and Features == "TSFresh" and FeatureCorrelation == "0.9"').shape

(9, 6)